In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import words

In [2]:
## DATA SETS ---------
df = pd.read_csv("spam.csv", encoding = "latin-1")
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

## TRANSFORMS
df.columns = ["Status", "Mensagem"]

print("Linhas x Colunas", df.shape, "\n") 
df.head() 


nltk.download("words")
set_words = set(words.words())

Linhas x Colunas (5572, 2) 

[nltk_data] Downloading package words to C:\Users\Luis
[nltk_data]     Carlos\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [3]:
df['Mensagem'] = df['Mensagem'].str.replace('\W', ' ') # Limpa a pontuação
df['Mensagem'] = df['Mensagem'].str.replace('subject', ' ') # Limpa o subject
df['Mensagem'] = df['Mensagem'].str.lower()  #Transforma tudo em letra pequena

df = df[["Mensagem", "Status"]]

df['Status']= df['Status'].map(str)
df['Status'] = df['Status'].str.replace('spam', '1') # Limpa o subject
df['Status'] = df['Status'].str.replace('ham', '0') # Limpa o subject

df.head()

,Mensagem,Status
0,go until jurong point crazy available only ...,0
1,ok lar joking wif u oni,0
2,free entry in 2 a wkly comp to win fa cup fina...,1
3,u dun say so early hor u c already then say,0
4,nah i don t think he goes to usf he lives aro...,0


In [4]:
def train(ficheiro):                                   # Função train - Recebe um ficheiro e cria um dicionário (vocabulary) das 
    for i in range(df.shape[0]):                          # Ciclo que corre todas as mensagens do ficheiro
        email = df.iloc[i, 0].split()                     # Mensagem separada por palavras

        for word in email:                                  # Ciclo que corre todas as palavras de uma mensagem
            if word.lower() not in vocabulary and word.lower() in set_words: # Verificação da existência da palavra no dicionário vocabulary e no set set_words
                vocabulary[word] = len(vocabulary)          # Atribuição de uma posição à palavra analisada no dicionário vocabulary
    generate(str(ficheiro))

In [5]:
def generate(ficheiro):                                # Função generate - Gera as matrizes data e verify
    global data                                             # Variável data definida como global
    data = np.zeros((df.shape[0], len(vocabulary)))       # Criação de uma matriz com número de linhas igual ao número de linhas do ficheiro e com número de colunas igual ao número de palavras (sem repetição) existentes no vocabulário
    global verify                                           # Variável verify definida como global
    verify = np.zeros(df.shape[0])                        # Criação de uma matriz com número de linhas igual ao número de linhas do ficheiro e com uma coluna

    for i in range(df.shape[0]):                          # Ciclo que corre todas as linhas do ficheiro
        email = df.iloc[i, 0].split()                     # Mensagem separada por palavras

        for word in email:                                  # Ciclo que corre todas as palavras de uma mensagem
            if word.lower() in vocabulary:                  # Verificação da existência da palavra no dicionário vocabulary
                data[i, vocabulary[word]] += 1              # Adição de valores à matriz data (+1 no índice da palavra que foi repetida)
                if df.iloc[i, 1] == 1:
                    verify[i] = 1                           # Adição de valores à matriz verify (adiciona 1 para spam)
                elif df.iloc[i, 1] == 0:
                    verify[i] = -1                          # Adição de valores à matriz verify (adiciona -1 para ham)

In [8]:
class Perceptron():
    def __init__(self, l_rate, n_iter):
        self.l_rate = l_rate
        self.n_iter = n_iter

    def classify(self, data, verify):
        self.errors_list = []
        probs = np.zeros(data.shape[0])

        for i in range(self.n_iter):
            errors = 0
            mail_count = 0
            for mail, target in zip(data, verify):
                prediction = self.predict(mail)
                update = self.l_rate * (target - prediction)
                if update != 0:
                    weight[1:] += update * mail
                    weight[0] += update
                errors += int(update != 0.0)
                probs[mail_count] = prediction
                mail_count += 1
            self.errors_list.append(errors)
        return probs

    def net_input(self, mail):
        return np.dot(mail, weight[1:]) + weight[0]

    def predict(self, mail):
        return np.where(self.net_input(mail) >= 0.0, 1, -1)


In [11]:
if __name__ == "__main__":
    vocabulary = {}
    train("Treino")
    weight = np.zeros(1 + data.shape[1])
    P = Perceptron(0.01, 20)
    P.classify(data, verify)

    generate("Treino")
    P = Perceptron(0, 1)
    classification = P.classify(data, verify)
 
    print("\nFicheiro\nNúmero de linhas analisadas: {}".format(data.shape[0]))
    print("Número de palavras existentes (sem repetição): {}".format(data.shape[1]))
    print("Número de mensagens classificadas como spam: {}".format(sum(verify == 1)))
    print("Número de mensagens classificadas como ham: {}".format(sum(verify == -1)))

    print("\nAlgoritmo Perceptrão\nNúmero de mensagens classificadas como spam: {}".format(sum(classification == 1)))
    print("Número de mensagens classificadas como ham: {}".format(sum(classification == -1)))
    print("\nPrecisão do algoritmo: {:0.3f} %".format(sum(classification == verify) / data.shape[1] * 100))



Ficheiro
Número de linhas analisadas: 5572
Número de palavras existentes (sem repetição): 3801
Número de mensagens classificadas como spam: 0
Número de mensagens classificadas como ham: 0

Algoritmo Perceptrão
Número de mensagens classificadas como spam: 2106
Número de mensagens classificadas como ham: 3466

Precisão do algoritmo: 0.000 %
